In [1]:
# Hands-on Data Cleaning & Quality Automation - Ques 1

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import os

# Step 1: Load the dataset
def load_data(filepath):
    _, file_ext = os.path.splitext(filepath)
    if file_ext == '.csv':
        df = pd.read_csv(filepath)
    elif file_ext == '.xlsx':
        df = pd.read_excel(filepath)
    else:
        raise ValueError("Unsupported file format")
    return df

# Step 2: Assess data quality
def check_data_quality(df):
    quality_report = {
        'missing_values': df.isnull().sum().to_dict(),
        'duplicates': df.duplicated().sum(),
        'total_rows': len(df),
        'memory_usage_MB': round(df.memory_usage().sum() / 1024**2, 2)
    }
    return quality_report

# Step 3: Standardize data types
def standardize_datatypes(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            try:
                df[column] = pd.to_datetime(df[column])
                print(f"Converted {column} to datetime")
            except (ValueError, TypeError):
                try:
                    df[column] = pd.to_numeric(df[column].str.replace('[^\d.]', '', regex=True))
                    print(f"Converted {column} to numeric")
                except:
                    pass
    return df

# Step 4: Handle missing values
def handle_missing_values(df):
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns

    if len(numeric_cols) > 0:
        num_imputer = SimpleImputer(strategy='median')
        df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])

    if len(categorical_cols) > 0:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])

    return df

# Step 5: Detect and handle outliers
def remove_outliers(df):
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    outliers_removed = {}

    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)].shape[0]
        df[col] = df[col].clip(lower=lower, upper=upper)
        if outliers > 0:
            outliers_removed[col] = outliers

    return df, outliers_removed

# Step 6: Validate the results
def validate_cleaning(df):
    report = {
        'missing_values_remaining': df.isnull().sum().sum(),
        'duplicates_remaining': df.duplicated().sum(),
        'data_types': df.dtypes.to_dict()
    }
    return report

# Main execution
if _name_ == "_main_":
    # Replace 'your_dataset.csv' with your actual dataset path
    filepath = 'your_dataset.csv'
    df = load_data(filepath)

    print("Initial Data Quality Report:")
    print(check_data_quality(df))

    df = standardize_datatypes(df)
    df = handle_missing_values(df)
    df, outliers_removed = remove_outliers(df)

    print("\nOutliers Removed:")
    print(outliers_removed)

    print("\nFinal Validation Report:")
    print(validate_cleaning(df))

    # Optionally, save the cleaned data
    df.to_csv('cleaned_data.csv', index=False)

NameError: name '_name_' is not defined